In [1]:
import numpy as np
import anacal
import galsim
import numpy.lib.recfunctions as rfn

In [2]:
# Setups
norder = 6
det_nrot = 4
snr_min = 12
r2_min = 0.1
c0 = 5.0
mag_zero = 30
sigma_arcsec = 0.52
pixel_scale = 0.2
pthres=0.12

ncol = 24

mm = np.abs(np.random.randn(1000, ncol) * 10)
nn = np.random.randn(1000, ncol) * 4
cov_matrix = np.abs(2.0 + np.random.randn(ncol, ncol))

In [3]:
cov_obj = anacal.fpfs.table.Covariance(
    norder=norder,
    det_nrot=det_nrot,
    array=cov_matrix,
    mag_zero=mag_zero,
    pixel_scale=pixel_scale,
    sigma_arcsec=sigma_arcsec,
)
meas_obj = anacal.fpfs.ctask.CatalogTask(
    norder=norder,
    det_nrot=det_nrot,
    cov_matrix=cov_obj,
)
meas_obj.update_parameters(
    snr_min=snr_min,
    r2_min=r2_min,
    c0=c0,
    pthres=0.12,
)

print(len(meas_obj.meas_task.colnames))
print(len(meas_obj.det_task.colnames))

12
24


In [4]:
meas_obj.det_task.colnames

['m00',
 'm20',
 'm22c',
 'm22s',
 'm40',
 'm42c',
 'm42s',
 'm44c',
 'm44s',
 'm60',
 'm64c',
 'm64s',
 'v0',
 'v1',
 'v2',
 'v3',
 'v0r1',
 'v1r1',
 'v2r1',
 'v3r1',
 'v0r2',
 'v1r2',
 'v2r2',
 'v3r2']

In [5]:
src_1 = anacal.fpfs.table.Catalog(
    norder=norder,
    det_nrot=det_nrot,
    array=mm,
    noise=nn,
    mag_zero=mag_zero,
    pixel_scale=pixel_scale,
    sigma_arcsec=sigma_arcsec,
)
nshapelets = len(meas_obj.meas_task.colnames)
src_2 = anacal.fpfs.table.Catalog(
    norder=norder,
    array=mm[:, :nshapelets],
    noise=nn[:, :nshapelets],
    mag_zero=mag_zero,
    pixel_scale=pixel_scale,
    sigma_arcsec=sigma_arcsec,
)
out2 = meas_obj.run(catalog=src_1, catalog2=src_2)

In [6]:
out2[0]

(0.13567892, 0.91741946, 0.15882597, -0.572125, 0.15854847, -0.60671979, 0.12016562, -0.92139074, 1., 0., 0., 91.86259593, 0.13567892, 0.91741946, 0.15882597, -0.572125, 0.15854847, -0.60671979, 0.12016562, -0.92139074)

In [7]:
print(len(meas_obj.meas_task.colnames))
print(len(meas_obj.det_task.colnames))
dtype = [(nn, "f8") for nn in meas_obj.det_task.colnames]
mm_st =  rfn.unstructured_to_structured(mm[0:1], dtype=dtype)
nn_st =  rfn.unstructured_to_structured(nn[0:1], dtype=dtype)

12
24


In [11]:
dm_dg = anacal.fpfs.measure_shapelets_dg(mm_st, nn_st)
ell = anacal.fpfs.measure_fpfs_shape(c0* cov_obj.std_m00, mm_st, dm_dg)

In [12]:
ell

array([(0.13567892, 0.91741946, 0.15882597, -0.572125, 0.15854847, -0.60671979, 0.12016562, -0.92139074)],
      dtype=[('e1', '<f8'), ('e1_g1', '<f8'), ('e2', '<f8'), ('e2_g2', '<f8'), ('q1', '<f8'), ('q1_g1', '<f8'), ('q2', '<f8'), ('q2_g2', '<f8')])

In [10]:
mm_st

array([(24.61560848, 2.56267434, 4.22300944, 4.94346182, 13.28778653, 4.93482465, 3.74015749, 3.87098687, 11.42966139, 13.42636968, 8.05655023, 10.31636232, 3.42685322, 3.65265656, 3.44066776, 8.01732341, 17.49581553, 3.41494125, 3.74577864, 7.45798746, 4.02653116, 10.54748648, 21.24840753, 9.22624033)],
      dtype=[('m00', '<f8'), ('m20', '<f8'), ('m22c', '<f8'), ('m22s', '<f8'), ('m40', '<f8'), ('m42c', '<f8'), ('m42s', '<f8'), ('m44c', '<f8'), ('m44s', '<f8'), ('m60', '<f8'), ('m64c', '<f8'), ('m64s', '<f8'), ('v0', '<f8'), ('v1', '<f8'), ('v2', '<f8'), ('v3', '<f8'), ('v0r1', '<f8'), ('v1r1', '<f8'), ('v2r1', '<f8'), ('v3r1', '<f8'), ('v0r2', '<f8'), ('v1r2', '<f8'), ('v2r2', '<f8'), ('v3r2', '<f8')])

In [ ]:
anacal.fpfs.measure_fpfs_shear(
    C0=c0 * cov_obj.std_m00, 
    sigma_v=cov_obj.sigma_v,
    pcut=pcut, pthres=pthres,
    m00_min, sigma_m00,
    r2_min, sigma_r2",
    x=mm_st, y=nn_st,
)

12
24


TypeError: measure_fpfs_shape(): incompatible function arguments. The following argument types are supported:
    1. (C0: float, x: anacal::FpfsShapelets, x_dg: anacal::FpfsShapeletsResponse) -> anacal::FpfsShape
    2. (C0: float, x: anacal::FpfsDetect, x_dg: anacal::FpfsShapeletsResponse) -> anacal::FpfsShape

Invoked with: kwargs: C0=3.339548835394703, x=array((18.12902198, 3.86616865, 14.00935768, 4.88683147, 10.50202416, 3.58906992, 5.1365689, 6.20858287, 0.24820708, 0.85894457, 3.5630947, 0.03117425, 17.7861014, 13.57698456, 6.88896355, 3.18191733, 3.56082333, 18.56357802, 2.2567321, 10.63390959, 1.3178675, 7.64266925, 13.7216184, 5.49476592),
      dtype=[('m00', '<f8'), ('m20', '<f8'), ('m22c', '<f8'), ('m22s', '<f8'), ('m40', '<f8'), ('m42c', '<f8'), ('m42s', '<f8'), ('m44c', '<f8'), ('m44s', '<f8'), ('m60', '<f8'), ('m64c', '<f8'), ('m64s', '<f8'), ('v0', '<f8'), ('v1', '<f8'), ('v2', '<f8'), ('v3', '<f8'), ('v0r1', '<f8'), ('v1r1', '<f8'), ('v2r1', '<f8'), ('v3r1', '<f8'), (

In [56]:
mm_st

array((18.12902198, 3.86616865, 14.00935768, 4.88683147, 10.50202416, 3.58906992, 5.1365689, 6.20858287, 0.24820708, 0.85894457, 3.5630947, 0.03117425, 17.7861014, 13.57698456, 6.88896355, 3.18191733, 3.56082333, 18.56357802, 2.2567321, 10.63390959, 1.3178675, 7.64266925, 13.7216184, 5.49476592),
      dtype=[('m00', '<f8'), ('m20', '<f8'), ('m22c', '<f8'), ('m22s', '<f8'), ('m40', '<f8'), ('m42c', '<f8'), ('m42s', '<f8'), ('m44c', '<f8'), ('m44s', '<f8'), ('m60', '<f8'), ('m64c', '<f8'), ('m64s', '<f8'), ('v0', '<f8'), ('v1', '<f8'), ('v2', '<f8'), ('v3', '<f8'), ('v0r1', '<f8'), ('v1r1', '<f8'), ('v2r1', '<f8'), ('v3r1', '<f8'), ('v0r2', '<f8'), ('v1r2', '<f8'), ('v2r2', '<f8'), ('v3r2', '<f8')])

In [50]:
len(dtype)

24

In [51]:
dtype

[('m00', 'f8'),
 ('m20', 'f8'),
 ('m22c', 'f8'),
 ('m22s', 'f8'),
 ('m40', 'f8'),
 ('m42c', 'f8'),
 ('m42s', 'f8'),
 ('m44c', 'f8'),
 ('m44s', 'f8'),
 ('m60', 'f8'),
 ('m64c', 'f8'),
 ('m64s', 'f8'),
 ('v0', 'f8'),
 ('v1', 'f8'),
 ('v2', 'f8'),
 ('v3', 'f8'),
 ('v0r1', 'f8'),
 ('v1r1', 'f8'),
 ('v2r1', 'f8'),
 ('v3r1', 'f8'),
 ('v0r2', 'f8'),
 ('v1r2', 'f8'),
 ('v2r2', 'f8'),
 ('v3r2', 'f8')]